In [1]:
import cv2
import numpy as np
import json, os
import tensorflow as tf
import re
import utils

2024-02-05 12:26:38.644346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
%cd ..

In [21]:
def prepare(filename):
    img = cv2.imread(filename,0)
    height,width = img.shape
    return [[img.item(j,i) != 0 for j in range(height)] for i in range(width)]

def get_bounds(pixels):
    width = len(pixels)
    height = len(pixels[0])
    annotations = {
        "xmin": width,
        "xmax": 0,
        "ymin": height,
        "ymax": 0
    }
    for i in range(width):
        for j in range(height):
            if pixels[i][j] and i < annotations["xmin"]:
                annotations["xmin"] = i
            elif pixels[i][j] and i > annotations["xmax"]:
                annotations["xmax"] = i
            if pixels[i][j] and j < annotations["ymin"]:
                annotations["ymin"] = j
            elif pixels[i][j] and j > annotations["ymax"]:
                annotations["ymax"] = j
    annotations["xmin"] -= 1
    annotations["xmax"] += 1
    annotations["ymin"] -= 1
    annotations["ymax"] += 1
    return annotations

def annotate(filename:str, class_name, class_id):
    pixels = prepare(filename)
    width = len(pixels)
    height = len(pixels[0])
    info = get_bounds(pixels)
    expr = re.compile("\\.[^\\.]*$")
    f_format = expr.search(filename).group(0)
    with tf.io.gfile.GFile(filename, 'rb') as fid:
        encoded_image = fid.read()
    return tf.train.Example(features=tf.train.Features(feature={
        'image/height': utils.int64_feature(height),
        'image/width': utils.int64_feature(width),
        'image/filename': utils.bytes_feature(filename.split('/')[-1].encode()),
        'image/source_id': utils.bytes_feature(filename.encode()),
        'image/encoded': utils.bytes_feature(encoded_image),
        'image/format': utils.bytes_feature(f_format[1:].encode()),
        'image/object/bbox/xmin': utils.float_list_feature([info["xmin"] / width]),
        'image/object/bbox/xmax': utils.float_list_feature([info["xmax"] / width]),
        'image/object/bbox/ymin': utils.float_list_feature([info["ymin"] / height]),
        'image/object/bbox/ymax': utils.float_list_feature([info["ymax"] / height]),
        'image/object/class/text': utils.bytes_list_feature([class_name.encode()]),
        'image/object/class/label': utils.int64_list_feature([class_id]),
    }))

In [22]:
def annotate_dir(dirname):
    expr = re.compile("^\\d+")
    class_id = int(expr.search(dirname).group(0))
    class_name = dirname
    files = os.listdir(os.path.join("bricks", dirname))
    files = filter(lambda z: z != "annotations.json", files)
    records = [*map(lambda z: annotate(os.path.join("bricks", dirname, z), class_name, class_id), files)]
    for index, record in enumerate(records):
        with tf.io.TFRecordWriter(os.path.join("workspace", "data", f"{class_name}-{index}.record")) as writer:
            writer.write(record.SerializeToString())
    return {
        "id": class_id,
        "name": class_name
    }

In [23]:
annotate_dir('2357 brick corner 1x2x2')

TypeError: '2357' has type str, but expected one of: int

In [12]:
record = annotate('bricks/2357 brick corner 1x2x2/2357 brick corner 1x2x2 000L.png', '2357 brick corner', 2357)

In [14]:
writer = tf.io.TFRecordWriter("test.record")
writer.write(record.SerializeToString())
writer.close()

In [39]:
img = cv2.imread('bricks/2357 brick corner 1x2x2/2357.png', 0)

In [40]:
img.shape

(350, 400)

In [9]:
print(img[200,200])

134


In [43]:
height,width = img.shape
color = [[img.item(j,i) != 0 for j in range(height)] for i in range(width)]

In [22]:
annotations = {
    "xmin": width,
    "xmax": 0,
    "ymin": height,
    "ymax": 0
}

In [48]:
color[350][0]

False

In [56]:
def prepare(filename):
    img = cv2.imread(filename,0)
    height,width = img.shape
    return [[img.item(j,i) != 0 for j in range(height)] for i in range(width)]

In [72]:
# @jit(forceobj=True, looplift=True)
def get_bounds(pixels):
    width = len(pixels)
    height = len(pixels[0])
    annotations = {
        "xmin": width,
        "xmax": 0,
        "ymin": height,
        "ymax": 0
    }
    for i in range(width):
        for j in range(height):
            if pixels[i][j] and i < annotations["xmin"]:
                annotations["xmin"] = i
            elif pixels[i][j] and i > annotations["xmax"]:
                annotations["xmax"] = i
            if pixels[i][j] and j < annotations["ymin"]:
                annotations["ymin"] = j
            elif pixels[i][j] and j > annotations["ymax"]:
                annotations["ymax"] = j
    annotations["xmin"] -= 1
    annotations["xmax"] += 1
    annotations["ymin"] -= 1
    annotations["ymax"] += 1
    return annotations

In [69]:
get_bounds(color)

{'xmin': 74, 'xmax': 322, 'ymin': 86, 'ymax': 312}

In [24]:
annotations

{'xmin': 75, 'xmax': 321, 'ymin': 112, 'ymax': 336}

In [55]:
%timeit get_bounds(color)

28.9 ms ± 89.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit prepare('bricks/3003 brick 2x2/3003 brick 2x2 000L.png')

21.3 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
len(pixels[0])

400

In [73]:
%timeit get_bounds(pixels)

28.7 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
def annotate(filename):
    pixels = prepare(filename)
    width = len(pixels)
    height = len(pixels[0])
    info = get_bounds(pixels)
    return {
        "filename": filename,
        "width": width,
        "height": height,
        "bounds": info
    }

In [76]:
%timeit annotate('bricks/3003 brick 2x2/3003 brick 2x2 000L.png')

49.3 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [78]:
json.dumps(annotate('bricks/3003 brick 2x2/3003 brick 2x2 000L.png'))

'{"filename": "bricks/3003 brick 2x2/3003 brick 2x2 000L.png", "width": 400, "height": 400, "bounds": {"xmin": 94, "xmax": 325, "ymin": 67, "ymax": 330}}'